In [1]:
import pandas as pd

### Load Data from Source Files

In [2]:

df_recipes = pd.read_csv("../data/recipes.csv")
df_interactions = pd.read_csv("../data/interactions.csv")

In [3]:
interactions_sum = df_interactions["rating"].sum()
interactions_count = df_interactions["rating"].count()

df_recipe_ids = df_recipes[["id"]]
df_mean_ratings = df_recipe_ids.copy()[["id"]]
df_mean_ratings["mean_rating"] = (interactions_sum/interactions_count)

In [4]:
df_merged = pd.merge(
    df_interactions,
    df_recipes,
    left_on=["recipe_id"],
    right_on=["id"], how="left"
    )[["id", "name", "rating"]]

In [5]:
df_ratings_count = df_merged.groupby(["id"]) \
    .count()[["rating"]] \
    .reset_index() \
    .rename(columns={"rating": "ratings_count"})

In [6]:
df_ratings_average = df_merged.groupby(["id"]) \
    .mean()[["rating"]] \
    .reset_index() \
    .rename(columns={"rating": "ratings_average"})

In [7]:
df_tmp_one = pd.merge(df_mean_ratings, df_ratings_count, on="id")
df_tmp_two = pd.merge(df_tmp_one, df_ratings_average, on="id")
df_tmp_two["min_ratings_count"] = df_tmp_two["ratings_count"] \
    .quantile(1 - (10000/df_tmp_two["ratings_count"].count()))

def weighted_average(row):
    ratings_count = row["ratings_count"]
    ratings_average = row["ratings_average"]
    min_ratings_count = row["min_ratings_count"]
    mean_rating = row["mean_rating"]
    return ((ratings_average * ratings_count) + (mean_rating * min_ratings_count))/(ratings_count + min_ratings_count)

df_tmp_two["ratings_weighted_average"] = df_tmp_two.apply(lambda r : weighted_average(r), axis=1)

In [8]:
df_recipe_popularity = df_tmp_two.sort_values(["ratings_weighted_average"], ascending=False) \
    .reset_index(drop=True)
df_recipe_popularity = pd.merge(df_recipe_popularity, df_recipes, on="id")[[
        "id",
        "name",
        "mean_rating",
        "ratings_count",
        "ratings_average",
        "min_ratings_count",
        "ratings_weighted_average"
    ]]

In [9]:
df_recipe_popularity.to_csv("../data/recipe_popularity.csv", index=False)